# Pressure Data

We can look at the raw pressure data from the BMP180 seonsor, with only a naïve correction to local pressure-altitude of the launch site:

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665

columns = loadtxt("../fc-data/BMP1.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
pressure = columns[2]
temperature = columns[3]

t_0 = 117853569585227

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

In [ ]:
cal_AC1 = 408
cal_AC2 = -72
cal_AC3 = -14383
cal_AC4 = 32741
cal_AC5 = 32757
cal_AC6 = 23153
cal_B1 = 6190
cal_B2 = 4
cal_MB = -32767
cal_MC = -8711
cal_MD = 2868

mode = 4


pressures = []
last_raw = 556878
for i, t in enumerate(timestamp):

    UT = int(temperature[i])
    UP = int(pressure[i])
    
    msb =  (UP & 0xff000000) >> (8*3)
    lsb =  (UP & 0x00ff0000) >> (8*2)
    xlsb = (UP & 0x0000ff00) >> (8*1)

    raw = ((msb << 16) + (lsb << 8) + xlsb) >> (8 - mode)
    #print "Data: 0x%x" % UP
    #print "MSB: 0x%x, LSB: 0x%x, xLSB: 0x%x" % (msb, lsb, xlsb)
    #print "Raw: 0x%x" % raw
    
    # dump gaps
    if abs(raw - last_raw) > 10000:
        raw = last_raw
    pressures.append(raw*(85.7/556878.0))
    last_raw = raw
    
    X1 = ((UT - cal_AC6) * cal_AC5) >> 15
    X2 = (cal_MC << 11) / (X1 + cal_MD)
    
    B5 = X1 + X2
    #print 'B5 = {0}'.format(B5)
    
    # Pressure Calculations
    B6 = B5 - 4000
    #print 'B6 = {0}'.format(B6)
    
    X1 = (cal_B2 * (B6 * B6) >> 12) >> 11
    X2 = (cal_AC2 * B6) >> 11
    X3 = X1 + X2
    B3 = (((cal_AC1 * 4 + X3) << mode) + 2) / 4
    #print 'B3 = {0}'.format(B3)
    
    X1 = (cal_AC3 * B6) >> 13
    X2 = (cal_B1 * ((B6 * B6) >> 12)) >> 16
    X3 = ((X1 + X2) + 2) >> 2
    B4 = (cal_AC4 * (X3 + 32768)) >> 15
    #print 'B4 = {0}'.format(B4)
  
    B7 = (UP - B3) * (50000 >> mode)
    #print 'B7 = {0}'.format(B7)
    
    if B7 < 0x80000000:
        p = (B7 * 2) / B4
    else:
        p = (B7 / B4) * 2

    X1 = (p >> 8) * (p >> 8)
    X1 = (X1 * 3038) >> 16
    X2 = (-7357 * p) >> 16
    p = p + ((X1 + X2 + 3791) >> 4)
    
    #pressures.append(p)
    #print 'Pressure {0} kPa'.format(p/1000.0)

columns = loadtxt("uncalibrated_integrated_altitude.csv", delimiter=',', unpack=True)
alt_time = columns[0]
imualt   = columns[1]

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"BMP180 Pressure Sensor Uncalibrated")
plt.ylabel(r"Pressure [kPa]")
plt.xlabel(r"Mission Elapsed Time [s]")
ax1.plot(timestamp, pressures, alpha=0.75, label="Pressure Sensor")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Altitude [km]")
ax2.plot(alt_time, divide(imualt, 1000.0), 'k-', alpha=0.18, label="IMU Integrated Altitude")
ax2.grid(b=False)

ax1.set_xlim([-5,42])
ax2.set_xlim([-5,42])
plt.legend()
plt.show()

The Pressure goes down and the rocket goes up, as expected.

## Temperature

The BMP180 also has a temperature sensor onboard to do temperature corrections for pressure value. The temperature of the device appears to have been relativly constant during the vertical flight.

In [ ]:
temps = []
for i, t in enumerate(timestamp):
    UT = int(temperature[i])

    X1 = ((UT - cal_AC6) * cal_AC5) >> 15
    X2 = (cal_MC << 11) / (X1 + cal_MD)
    B5 = X1 + X2
    temp = ((B5 + 8) >> 4) / 10.0
    temps.append(temp)
    
          
ax = plt.figure(figsize=(16,6))
plt.title(r"BMP180 Pressure Sensor Uncalibrated Temperature")
plt.ylabel(r"Temperature [${}^0$C]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp, temps, alpha=0.75)
ax.axes[0].set_xlim([-5,42])
plt.show()

## Altitude

Even with a rough guess of the pressure, we can apply an exponetial atmosphere and see what altitude the pressure sensor suggests we went to:

In [ ]:
sealevel_pa=101.325
altitudes = []
for p in pressures:
    altitude = 44330.0 * (1.0 - pow(p / sealevel_pa, (1.0/5.255)))
    altitude = altitude - 1390
    altitudes.append(altitude)

    
ax = plt.figure(figsize=(16,6))
plt.title(r"BMP180 Pressure Sensor Uncalibrated Altitude (Exponental Atmosphere, No Temperature Correction)")
plt.ylabel(r"Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp, divide(altitudes, 1000.0), alpha=0.75, label="Pressure Sensor")
plt.plot(alt_time, divide(imualt, 1000.0), alpha=0.5, label="IMU Integrated Altitude")
ax.axes[0].set_xlim([-5,42])
#ax.axes[0].set_ylim([200,0])
plt.legend()
plt.show()

The large diversion suggests a need for calibration. We will compare this result to other sensors on the flight later in this paper.